# 7장 이미지 인식의 은총알 CNN

**합성곱 신경망** 은 얀리쿤 교수가 소개하였다. 음성인식이나 자연어 처리 등에도 사용되고 활용성에서도 매우 뛰어난 성과를 보여주고 있다.

## 7.1 CNN 개념

CNN 모델은 기본적으로 **컨볼루션 계층(Convolution layer, 합성곱 계층)**과 **풀링 계층(pooling layer)**으로 구성된다.

몇 칸씩 움직일지 정하는 값을 **스트라이드**라고 한다.

입력층의 윈도우를 은닉층의 뉴런 하나로 압축할 때, 컨볼루션 계층에서는 윈도우 크기만큼의 가중치와 1개의 편향을 적용한다. 윈도우 크기가 3 $\times$ 3 이라면 3 $\times$ 3개의 가중치와 1개의 편향이 필요하다. 이 3 $\times$ 3개의 가중치와 1개의 편향을 **커널** 또는 **필터** 라고 하며, 이 커널은 해당 은닉층을 만들기 위한 모든 윈도우에 공통으로 적용된다.

이는 CNN의 가장 중요한 특징이다. 가령 입력층이 28 $\times$ 28개일때, 기본 신경망으로 모든 뉴런을 연결한다면 784개의 가중치를 찾아내야 하겠지만, 컨볼루션 계층에서는 3 $\times$ 3개인 9개의 가중치만 찾아내면 된다. 따라서 계산량이 매우 적어져 학습이 더 빠르고 효율적으로 이루어진다.

물론 이 경우 부족할 수 있으니 보통 커널을 여러개 사용한다. 커널의 크기나 개수 역시 하이퍼파라미터의 하나로써 분석하고자 하는 내용에 따라 커널의 개수를 정하는 일 역시 매우 중요하다.

## 7.2 모델 구현하기

In [2]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


1. 앞에서는 모델의 입력값을 28 $\times$ 28 짜리 차원 하나로 구성했지만, CNN 모델에서는 2차원 평면으로 구성하므로 다음처럼 조금 직관적인 형태로 구성한다. 그리고 마지막 차원인 1은 특징의 개수로, MNIST 데이터는 회색조 이미지라 채널에 색상이 한개뿐이므로 1을 사용한다. 그리고 출력값이 10개의 분류와, 드롭아웃을 위한 keep_prob 플레이스 홀더도 정의한다.

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

3. 3 $\times$ 3 크기의 커널을 가진 컨볼루션 계층을 만든다. tf.nn.conv2d 함수를 사용하면 간단하게 구할 수 있다.

In [4]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

입력층 X와 첫 번째 계층의 W1을 가지고, 오른쪽과 아래쪽으로 한 칸씩 움직이는 32개의 커널을 가진 컨볼루션 계층을 만들겠다는 코드이다. 여기서 padding='SAME'은 커널 슬라이딩 시 이미지의 가장 외각으로 한칸 밖으로 움직여서, 이미지의 테두리까지 정확하게 평가하게 한다. 그리고 tf.nn.relu 활성화 함수를 통해 컨볼루션 계층을 완성할 수 있다.

3. 풀링 계층을 만든다. 

In [5]:
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

이 코드는 컨볼루션 계층을 입력층으로 사용하고, 커널 크기를 2 $\times$ 2로 하는 풀링 계층을 만든다. strides = [1, 2, 2, 1] 값은 슬라이딩 시 두 칸씩 움직이겠다는 옵션이다.

4. 두번째 계층

In [7]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

3 $\times$ 3 크기의 커널 64개로 구성한 컨볼루션 계층과 2 $\times$ 2 크기의 풀링 계층으로 구성하였다. 두 번째 컨볼루션 계층의 커널인 W2 변수의 구성은 [3, 3, 32, 64]이다. 32는 앞서 구성한 첫 번째 컨볼루션 계층의 커널 개수이다. 이는 출력층의 개수이고, 첫 번째 컨볼루션 계층이 찾아낸 이미지의 특징 개수라고 할 수 있다.

5. 추출한 특징들을 이용해 10개의 분류를 만들어내는 계층을 구성해보자.

In [8]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

10개의 분류는 1차원 배열이므로 차우너을 줄이는 단계를 거쳐야 한다. 먼저 직전의 풀링 계층의 크기가 7 $\times$ 7 $\times$ 64이므로 먼저, tf.reshape 함수를 이용해 7 $\times$ 7 $\times$ 64 크기의 1차원 계층으로 만들고, 이 배열 전체를 최종 출력값의 중간 형태인 256개의 뉼너으로 연결하는 신경망을 만들어 준다. 이처럼 인접한 계층의 모든 뉴런과 상호 연결된 계층을 **완전 연결 계층**이라 한다. 추가로 과적합을 방지해주는 드랍아웃을 사용한다.

6. 모델 구성의 마지막으로 직전의 은닉층인 L3의 출력값 256개를 받아 최종 출력값인 0~9 레이블을 갖는 10개의 출력값을 만든다.

In [9]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

7. 손실 함수와 AdamOptimizer를 이용한 최적화 함수

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

8. 결과 확인. 입력값을 전달할때 MNIST 데이터를 28 $\times$ 28 형태로 재구성

```
batch_xs.reshape(-1, 28, 28, 1)
mnist.test.images.reshape(-1, 28, 28, 1)
```

In [12]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        
        total_cost += cost_val
        
    print ('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))
    
print ('최적화 완료!')

Epoch: 0001 Avg. cost =  0.192
Epoch: 0002 Avg. cost =  0.076
Epoch: 0003 Avg. cost =  0.060
Epoch: 0004 Avg. cost =  0.054
Epoch: 0005 Avg. cost =  0.056
Epoch: 0006 Avg. cost =  0.051
Epoch: 0007 Avg. cost =  0.050
Epoch: 0008 Avg. cost =  0.051
Epoch: 0009 Avg. cost =  0.042
Epoch: 0010 Avg. cost =  0.049
Epoch: 0011 Avg. cost =  0.044
Epoch: 0012 Avg. cost =  0.043
Epoch: 0013 Avg. cost =  0.038
Epoch: 0014 Avg. cost =  0.047
Epoch: 0015 Avg. cost =  0.045
최적화 완료!


In [13]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도:', sess.run(accuracy,
                         feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob: 1}))
        

정확도: 0.9884


## 전체 코드

In [5]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

################
# 신경망 모델 구성 #
################

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

################
# 신경망 모델 학습 #
################

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob:0.6})
        
        total_cost += cost_val
        
    print ('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))
    
print ('최적화 완료!')

###########
# 결과 확인 #
##########

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도:', sess.run(accuracy,
                         feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob: 1}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Avg. cost =  0.366
Epoch: 0002 Avg. cost =  0.124
Epoch: 0003 Avg. cost =  0.089
Epoch: 0004 Avg. cost =  0.074
Epoch: 0005 Avg. cost =  0.061
Epoch: 0006 Avg. cost =  0.051
Epoch: 0007 Avg. cost =  0.045
Epoch: 0008 Avg. cost =  0.040
Epoch: 0009 Avg. cost =  0.035
Epoch: 0010 Avg. cost =  0.032
최적화 완료!
정확도: 0.9887


## 7.3 고수준 API

다음 4줄의 코드를 두줄로 줄일 수 있다.

```py
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
```

이를 tf.layers 모듈을 이용하면 다음의 두 줄로 줄일 수 있다.

```py
L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
```

2. 완전 연결 계층을 만드는 부분의 이전 코드는 다음과 같다

```py
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
```

이를 tf.layers 모듈을 사용하면 다음처럼 간단하게 작성할 수 있다.

```
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
```

## 전체 코드

In [14]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

################
# 신경망 모델 구성 #
################

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3])
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

################
# 신경망 모델 학습 #
################

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob:0.6})
        
        total_cost += cost_val
        
    print ('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))
    
print ('최적화 완료!')

###########
# 결과 확인 #
##########

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도:', sess.run(accuracy,
                         feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob: 1}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Avg. cost =  0.187
Epoch: 0002 Avg. cost =  0.050
Epoch: 0003 Avg. cost =  0.033
Epoch: 0004 Avg. cost =  0.022
Epoch: 0005 Avg. cost =  0.017
Epoch: 0006 Avg. cost =  0.014
Epoch: 0007 Avg. cost =  0.011
Epoch: 0008 Avg. cost =  0.007
Epoch: 0009 Avg. cost =  0.007
Epoch: 0010 Avg. cost =  0.012
Epoch: 0011 Avg. cost =  0.004
Epoch: 0012 Avg. cost =  0.004
Epoch: 0013 Avg. cost =  0.012
Epoch: 0014 Avg. cost =  0.004
Epoch: 0015 Avg. cost =  0.004
최적화 완료!
정확도: 0.9887


## 7.4 더 보기

텐서플로를 사용한다면 구글이 제공하는 Cloud ML을 이용해 매우 편하게 학습을 시킬 수 있다. 학습 뿐 아니라 학습된 모델을 Cloud ML에 올려 쉽고 편하게 예측 서비스에 사용할 수도 있다.